### Загрузка данных

In [18]:
import numpy as np
import pandas as pd

X = pd.read_csv('./assets/features.csv', index_col='match_id');

### Выделяем "признаки из будущего" и удаляем их из выборки

In [19]:
future_feature_names = [
        'duration' 
        , 'radiant_win'
        , 'tower_status_radiant'
        , 'tower_status_dire'
        , 'barracks_status_radiant'
        , 'barracks_status_dire'
    ]
future_features = X[future_feature_names]
for future_feature_name in future_feature_names:
    del X[future_feature_name]

y = future_features['radiant_win']

### Заполняем пропуски в данных

In [20]:
X_nona = X.fillna(0)

### Создаём генератор разбиений для кросс-валидации

In [21]:
from sklearn.cross_validation import KFold
kfold = KFold(n=len(X), n_folds=5, shuffle=True, random_state=241)

### 1.1. Ищем и выводим список колонок, которые имеют пропуски в значениях

In [22]:
X_len = len(X)
column_values_counts = ([column, X[column].count()] for column in X.columns)
columns_with_missing_values = [col_count[0] for col_count in column_values_counts if col_count[1] != X_len]
print 'Columns with missing values:'
print '\n'.join(['* ' + str(x) for x in columns_with_missing_values])

Columns with missing values:
* first_blood_time
* first_blood_team
* first_blood_player1
* first_blood_player2
* radiant_bottle_time
* radiant_courier_time
* radiant_flying_courier_time
* radiant_first_ward_time
* dire_bottle_time
* dire_courier_time
* dire_flying_courier_time
* dire_first_ward_time


Итого, следующие колонки имеют пропуски в значениях:
* first_blood_time
* first_blood_team
* first_blood_player1
* first_blood_player2
* radiant_bottle_time
* radiant_courier_time
* radiant_flying_courier_time
* radiant_first_ward_time
* dire_bottle_time
* dire_courier_time
* dire_flying_courier_time
* dire_first_ward_time

### 1.1. Что могут означать пропуски в значениях (выберите любые две колонки)
Пропуски в значениях первой колонки (на самом деле, первых четырёх) означают, что "первая кровь" была не была пролита в течение рассматриваемого нами времени.

Пропуски в `radiant_bottle_time` объясняются тем, что не во всех матчах хотя бы один из игроков команд покупает эту вещь.

### 1.2. Как называется столбец с целевой переменной?
Столбец называется `radiant_win`
### 1.3. Обучите модель для 10, 20 и 30 деревьев. Как долго проводилась кросс-валидация? Какое качество получилось?

In [25]:
import time
import datetime
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score

for i in xrange(10, 40, 10):
    print 'Estimators:', i
    clf = GradientBoostingClassifier(n_estimators=i, random_state=242)
    start_time = datetime.datetime.now()
    mean_score = cross_val_score(clf, X_nona, y=y, cv=kfold, scoring='roc_auc').mean()
    print 'Score:', mean_score
    print 'Time:', datetime.datetime.now() - start_time

Estimators: 10
Score: 0.664387785223
Time: 0:00:35.293893
Estimators: 20
Score: 0.682853571415
Time: 0:01:09.926190
Estimators: 30
Score: 0.689496203941
Time: 0:01:45.207535


Итого:

* обучение 10 деревьев заняло 35 сек., среднее качество: 0.664387785223
* 10 деревьев - 1 мин 9 сек, среднее качество: 0.682853571415
* 30 деревьев - 1 мин 46 сек, среднее качество: 0.689496203941

### 1.4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что можно сделать, чтобы ускорить его обучение при увеличении количества деревьев?
Ответим в обратном порядке. Для того, чтобы ускорить обучение при увеличении количества деревьев, можно сделать следующие шаги:

* использовать параметр `n_jobs` у `cross_val_score`, позволяющий проводить обучение параллельно.
* определить полезность признаков и отбросить лишние с помощью параметра `clf.feature_importances_`
* уменьшить размерность данных, например, с помощью метода главных компонент
* ограничить глубину дерева с помощью параметра `max_depth` классификатора
* взять для обучения только часть выборки, например, воспользовавшись функцией `train_test_split`

Сделаем первое и проверим на 100 деревьях:

In [22]:
clf = GradientBoostingClassifier(n_estimators=100, random_state=242)
start_time = datetime.datetime.now()
mean_score = cross_val_score(clf, X_nona, y=y, cv=kfold, scoring='roc_auc', n_jobs=-1).mean()
print 'Score:', mean_score
print 'Time:', datetime.datetime.now() - start_time

Score: 0.706327055343
Time: 0:02:40.871921


Итого, обучение 100 деревьев заняло 2 минуты 40 секунд, среднее качество: 0.706327055343, то есть, выше, но ненамного. Целесообразность повышения количества деревьев зависит от параметров задачи, например, того, какая производительность алгоритма считается приемлимой.

# 2. Логистическая регрессия

### Загрузка и подготовка данных, импорты и новый подход к измерению времени

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score

import time
import datetime
import numpy as np
import pandas as pd
import re


class TimeEstimator:
    def __init__(self):
        pass

    def __enter__(self):
        self.start_time = datetime.datetime.now()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print 'Elapsed time', datetime.datetime.now() - self.start_time

print '--> Preparing data...'
with TimeEstimator():
    X = pd.read_csv('./assets/features.csv', index_col='match_id')

    # remove "future" features
    future_feature_names = [
        'duration'
        , 'radiant_win'
        , 'tower_status_radiant'
        , 'tower_status_dire'
        , 'barracks_status_radiant'
        , 'barracks_status_dire'
    ]
    future_features = X[future_feature_names]
    for future_feature_name in future_feature_names:
        del X[future_feature_name]

    y = future_features['radiant_win']


--> Preparing data...
Elapsed time 0:00:00.732129


### Набор функций, используемый для обработки данныx

In [9]:
def get_cols_by_regex(all_cols, col_regex):
    r = re.compile(col_regex)
    col_match = np.vectorize(lambda x: x if bool(r.match(x)) else None)
    return [col for col in col_match(all_cols) if col is not None and col != 'None']


# returns numpy array
def fill_na_and_scale_features(x):
    print 'Scaling features...'
    x_nona = x.fillna(0)
    with TimeEstimator():
        scaler = StandardScaler()
        return scaler.fit_transform(x_nona)


def build_data_with_pick(x):
    print '--> Building pick information...'
    with TimeEstimator():
        # search for heroes
        hero_cols = get_cols_by_regex(x.columns, '.*_hero$')
        heroes = set()
        for col in hero_cols:
            col_heroes = x[col].unique()
            for hero_number in col_heroes:
                heroes.add(hero_number)

        print '--> Number of heroes: ', len(heroes)

        # build pick information
        pick_arr = np.zeros((x.shape[0], max(heroes) + 1))
        for i, match_id in enumerate(x.index):
            pick_arr[i, 0] = match_id
            for p in xrange(5):
                pick_arr[i, x.ix[match_id, 'r%d_hero' % (p + 1)]] = 1
                pick_arr[i, x.ix[match_id, 'd%d_hero' % (p + 1)]] = -1

        # build pick column labels, create data frame
        new_cols = ['match_id'] + map(lambda col: 'hero_' + str(col), xrange(1, max(heroes) + 1))
        pick_df = pd.DataFrame(data=pick_arr, columns=new_cols, index=pick_arr[:, 0])

        # concatenate pick data frame to the main data
        x_pick = pd.concat([x, pick_df], axis=1)
        return x_pick

# Does cross-validation with optional search of the best value for parameter C
def cross_val_on(x, y, calc_c=False):
    x_scaled = fill_na_and_scale_features(x)
    print 'Doing cross-validation...'

    with TimeEstimator():
        grid = {
            'C': np.power(10.0, np.arange(-5, 6))
        }
        kfold = KFold(n=y.size, n_folds=5, shuffle=True, random_state=241)

        if calc_c:
            clf = LogisticRegression(penalty='l2', random_state=241)
            grid_search = GridSearchCV(clf, grid, cv=kfold, scoring='roc_auc', n_jobs=-1)
            grid_search.fit(x_scaled, y)
            best_score = 0
            best_c = 0
            for a in grid_search.grid_scores_:
                if a.mean_validation_score > best_score:
                    best_score = a.mean_validation_score
                    best_c = a.parameters['C']
            print 'Best mean score: ', best_score, 'C:', best_c
            return best_c
        else:
            # Know C=0.01 is the best
            C = 0.01
            clf = LogisticRegression(random_state=241, C=C)
            mean_score = cross_val_score(clf, x_scaled, y=y, cv=kfold, scoring='roc_auc', n_jobs=-1).mean()
            print 'Mean score: ', mean_score
            return C


### 2.1 Какое качество получилось у логистической регрессии над всеми исходными признаками? Чем можно объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [10]:
print '--> Training regression on raw data...'
cross_val_on(X, y, calc_c=False)


--> Training regression on raw data...
Scaling features...
Elapsed time 0:00:00.142610
Doing cross-validation...
Mean score:  0.716341465365
Elapsed time 0:00:05.305648
Best C:  0.01


Итого, получилось 0.716341465365, что примерно на 3.8% выше, чем у градиентного бустинга. Предположительно, разница объясняется неплохой линейной разделимостью выборки. Кросс-валидация по 5 блокам заняла 5 секунд. Время на 30 вершинах градиентного бустинга 1 мин 46 сек, т.е. примерно в 30 раз больше. Причём для логистической регрессии параметр n_jobs существенно на скорость не влияет.

### 2.2 Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем можно объяснить это изменение?

In [12]:
print '--> Training on non-categorial features...'
non_categ_cols = get_cols_by_regex(X.columns, '^(?!(.*_hero$|^lobby_type$)).*')
X_no_categ = X[non_categ_cols]
cross_val_on(X_no_categ, y, calc_c=False)

--> Training on non-categorial features...
Scaling features...
Elapsed time 0:00:00.130356
Doing cross-validation...
Mean score:  0.716400950653
Elapsed time 0:00:04.803799


0.01

Оценка качества: 0.716400950653, время: 4 сек. Получили небольшой прирост качества и скорости. Причиной увеличения качества является неправильное использование признаков, создававшее шум. Причиной прироста скорости является уменьшение размерности пространства признаков.

### 2.3, 2.4 Cколько различных идентификаторов героев существует в данной игре? Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

In [26]:
X_pick = build_data_with_pick(X)

print '--> Training on full data set with pick columns (just for fun)...'
cross_val_on(X_pick, y, calc_c=False)

# recalculate non-category columns due to changed data set
pick_non_categ_cols = get_cols_by_regex(X_pick.columns, '^(?!(.*_hero$|^lobby_type$)).*')
X_pick_no_categ = X_pick[pick_non_categ_cols]

print '--> Training on pick data set without categ columns...'
best_C = cross_val_on(X_pick_no_categ, y, calc_c=True)
print 'Best C: ', best_C


--> Building pick information...
--> Number of heroes:  108
Elapsed time 0:00:04.192704
--> Training on full data set with pick columns (just for fun)...
Scaling features...
Elapsed time 0:00:00.287462
Doing cross-validation...
Mean score:  0.751908999986
Elapsed time 0:00:10.148128
--> Training on pick data set without categ columns...
Scaling features...
Elapsed time 0:00:00.276818
Doing cross-validation...
Best mean score:  0.751969165698 C: 0.01
Elapsed time 0:01:38.514678
Best C:  0.01


Итого, в играх из файла участвовало 108 различных героев. Качество после добавления мешка слов составило 0.751969165698. Прирост качества связан с тем, что информация об участвующих в игре героях важна с точки зрения победы в игре, и поэтому добавление её в выборку в правильной форме улучшает линейную разделимость.

### 2.5 Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?
Для ответа на вопрос подготовим данные из выборки аналогичным описанному выше способом, удалив из них категориальные признаки и добавив "мешок слов" по героям. После этого используем найденное лучшее значение для константы C, чтобы обучить классификатор и сделать предсказание.

In [16]:
print '--> Fitting regression with best c...'
fit_clf = LogisticRegression(penalty='l2', C=best_C, random_state=241)
X_to_fit = fill_na_and_scale_features(X_pick_no_categ)
print 'Fitting...'
with TimeEstimator():
    fit_clf.fit(X_to_fit, y)

print '--> Loading and preparing test data...'
X_test = pd.read_csv('./assets/features_test.csv', index_col='match_id')
X_test_with_pick = build_data_with_pick(X_test)
X_test_pick_no_categ = X_test_with_pick[pick_non_categ_cols]
X_test_scaled = fill_na_and_scale_features(X_test_pick_no_categ)
print '--> Predicting...'
with TimeEstimator():
    predictions = fit_clf.predict_proba(X_test_scaled)[:, 1]
print '--> Min predicted value: ', predictions.min(), 'Max predicted value: ', predictions.max()

--> Training on pick data set without categ columns...
Scaling features...
Elapsed time 0:00:00.275432
Doing cross-validation...
Best mean score:  0.751969165698 C: 0.01
Elapsed time 0:01:42.926424
Best C:  0.01
--> Fitting regression with best c...
Scaling features...
Elapsed time 0:00:00.275856
Fitting...
Elapsed time 0:00:03.953320
--> Loading and preparing test data...
--> Building pick information...
--> Number of heroes:  108
Elapsed time 0:00:00.786651
Scaling features...
Elapsed time 0:00:00.050387
--> Predicting...
Elapsed time 0:00:00.015273
--> Min predicted value:  0.00859504107819 Max predicted value:  0.996403192335


Итого:

* минимальное значение: 0.00859504107819
* максимальное значение: 0.996403192335